In [3]:
import pandas as pd
bike_data = pd.read_csv('data/citibike-tripdata.csv',  sep=',')
bike_data.head()

,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,2018-09-01 00:00:05.2690,2018-09-01 00:27:20.6340,252.0,MacDougal St & Washington Sq,40.732264,-73.998522,366.0,Clinton Ave & Myrtle Ave,40.693261,-73.968896,25577,Subscriber,1980,1
1,2018-09-01 00:00:11.2810,2018-09-01 00:02:23.4810,314.0,Cadman Plaza West & Montague St,40.693830,-73.990539,3242.0,Schermerhorn St & Court St,40.691029,-73.991834,34377,Subscriber,1969,0
2,2018-09-01 00:00:20.6490,2018-09-01 00:55:58.5470,3142.0,1 Ave & E 62 St,40.761227,-73.960940,3384.0,Smith St & 3 St,40.678724,-73.995991,30496,Subscriber,1975,1
3,2018-09-01 00:00:21.7460,2018-09-01 00:07:38.5830,308.0,St James Pl & Oliver St,40.713079,-73.998512,3690.0,Park Pl & Church St,40.713342,-74.009355,28866,Subscriber,1984,2
4,2018-09-01 00:00:27.3150,2018-09-01 02:21:25.3080,345.0,W 13 St & 6 Ave,40.736494,-73.997044,380.0,W 4 St & 7 Ave S,40.734011,-74.002939,20943,Customer,1994,1


In [24]:
bike_dt = bike_data.copy()
#bike_dt.info()
bike_dt['start station id'].mode()

0    281.0
dtype: float64

In [31]:
bike_dt['bikeid'].mode()

0.77


In [38]:
#bike_dt['usertype'].value_counts(normalize=True)*100
mode_usertype = bike_dt['usertype'].mode()[0]
count_mode_user = bike_dt[bike_dt['usertype'] == mode_usertype].shape[0]
print(round(count_mode_user / bike_dt.shape[0], 2))

300000

In [43]:
male_bike = bike_dt[bike_dt['gender']==1].shape[0]
female_bike = bike_dt[bike_dt['gender']==0].shape[0]
print(max([male_bike, female_bike]))

41912


In [59]:
#bike_dt['start station id'].nunique() == bike_dt['end station id'].nunique()
#(bike_dt['starttime'].dt.year - bike_dt['birth year']).min()
#bike_dt['start station name'].value_counts()
#bike_dt['end station name'].mode()

0    West St & Chambers St
dtype: object

In [64]:
#bike_dt.info()
#bike_dt.drop(['start station id', 'end station id'], axis=1, inplace=True)
#bike_dt.shape[1]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 12 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   starttime                300000 non-null  datetime64[ns]
 1   stoptime                 300000 non-null  object        
 2   start station name       299831 non-null  object        
 3   start station latitude   300000 non-null  float64       
 4   start station longitude  300000 non-null  float64       
 5   end station name         299831 non-null  object        
 6   end station latitude     300000 non-null  float64       
 7   end station longitude    300000 non-null  float64       
 8   bikeid                   300000 non-null  int64         
 9   usertype                 300000 non-null  object        
 10  birth year               300000 non-null  int64         
 11  gender                   300000 non-null  int64         
dtypes: datetime64[ns

In [67]:
bike_dt['age'] = 2018 - bike_dt['birth year']
bike_dt.drop(['birth year'], axis = 1, inplace=True)
bike_dt[bike_dt['age']>60].shape[0]

11837

In [81]:
bike_dt['starttime'] = pd.to_datetime(bike_dt['starttime'])
bike_dt['stoptime'] = pd.to_datetime(bike_dt['stoptime'])
bike_dt['trip duration'] = (bike_dt['stoptime'] - bike_dt['starttime']).dt.seconds
round(bike_dt['trip duration'].mean())

1000

In [84]:
bike_dt['weekend'] = bike_dt['starttime'].apply(lambda x: 1 if x.dayofweek == 5 or x.dayofweek == 6 else 0)
bike_dt['weekend'].sum()

115135

In [91]:
def time_of_day(daytime):
    if 0 <= daytime <= 6:
        return 'night'
    elif 6 < daytime <= 12:
        return 'morning'
    elif 12 < daytime <= 18:
        return 'day'
    elif 18 < daytime <= 23:
        return 'evening'
    else:
        return 'else'

bike_dt['time_of_day'] = bike_dt['starttime'].dt.hour.apply(time_of_day)
day_ride = bike_dt[bike_dt['time_of_day'] == 'day'].shape[0]
night_ride = bike_dt[bike_dt['time_of_day'] == 'night'].shape[0]
round(day_ride/night_ride)

9